In [13]:
import pandas as pd 
import yfinance as yf

In [14]:
goo = yf.Ticker('GOOG')
# sacar la informacion historica de 1 año hacia atras
hist = goo.history(period="1y")
hist['Date']=hist.index
hist=hist.reset_index(drop=True)
hist # tenemos 8 columnas

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
0,102.970001,104.029999,101.855003,103.629997,64540100,0.0,0.0,2022-09-16 00:00:00-04:00
1,102.540001,104.019997,102.370003,103.849998,19738600,0.0,0.0,2022-09-19 00:00:00-04:00
2,102.879997,103.169998,101.120003,101.830002,24001700,0.0,0.0,2022-09-20 00:00:00-04:00
3,102.239998,103.489998,99.989998,100.010002,26596800,0.0,0.0,2022-09-21 00:00:00-04:00
4,99.449997,101.680000,99.410004,100.570000,21272700,0.0,0.0,2022-09-22 00:00:00-04:00
...,...,...,...,...,...,...,...,...
246,137.380005,138.264008,136.550003,137.740005,17180800,0.0,0.0,2023-09-11 00:00:00-04:00
247,137.130005,137.639999,135.929993,136.070007,15212900,0.0,0.0,2023-09-12 00:00:00-04:00
248,135.899994,137.699997,134.929993,137.500000,16394900,0.0,0.0,2023-09-13 00:00:00-04:00
249,138.389999,139.550003,137.059998,138.990005,19064600,0.0,0.0,2023-09-14 00:00:00-04:00


In [15]:
hist=hist.rename(columns={'Open':'O','High':'H','Low':'L','Close':'C','Volume':'V',\
                          'Dividends':'D','Stock Splits':'S', 'Date':'Dat'})
hist= hist.drop(columns=['Dat']) #Eliminacion de Fecha por desinteres
hist

,O,H,L,C,V,D,S
0,102.970001,104.029999,101.855003,103.629997,64540100,0.0,0.0
1,102.540001,104.019997,102.370003,103.849998,19738600,0.0,0.0
2,102.879997,103.169998,101.120003,101.830002,24001700,0.0,0.0
3,102.239998,103.489998,99.989998,100.010002,26596800,0.0,0.0
4,99.449997,101.680000,99.410004,100.570000,21272700,0.0,0.0
...,...,...,...,...,...,...,...
246,137.380005,138.264008,136.550003,137.740005,17180800,0.0,0.0
247,137.130005,137.639999,135.929993,136.070007,15212900,0.0,0.0
248,135.899994,137.699997,134.929993,137.500000,16394900,0.0,0.0
249,138.389999,139.550003,137.059998,138.990005,19064600,0.0,0.0


In [26]:
import psycopg2
try:
    conn= psycopg2.connect(
        host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com",
        dbname= "data-engineer-database",
        user="frnmn20_coderhouse",
        password= "3076WtdbVk",
        port= "5439"
    )
    print ("Connected to Redshift successfully!")
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [27]:
conn

<connection object at 0x110b902e0; dsn: 'user=frnmn20_coderhouse password=xxx dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439', closed: 0>

In [23]:
hist.head()

,O,H,L,C,V,D,S
0,102.970001,104.029999,101.855003,103.629997,64540100,0.0,0.0
1,102.540001,104.019997,102.370003,103.849998,19738600,0.0,0.0
2,102.879997,103.169998,101.120003,101.830002,24001700,0.0,0.0
3,102.239998,103.489998,99.989998,100.010002,26596800,0.0,0.0
4,99.449997,101.680000,99.410004,100.570000,21272700,0.0,0.0


In [28]:
from psycopg2.extras import execute_values
cur = conn.cursor()
table_name = "stock_prices"
columns = ["O", "H", "L", "C", "V", "D", "S"]
values = [tuple(x) for x in hist.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")


In [29]:
cur.close()
conn.close()